In [1]:
import h5py
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score, recall_score, fbeta_score
from tensorflow.keras.models import clone_model
import pathlib
from tqdm import tqdm
import sys
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
attack_vector='cache' #network
attack_vector= 'network'
machine_setting='dynamic_browser_version' # static_browser_version

root_path=f'/content/drive/MyDrive/Colab Notebooks/concept-drift-online/'
data_path=f'{root_path}/data/{machine_setting}/{attack_vector}'
model_0_train_path=f'2021_08_27/train.h5'


In [6]:
os.getcwd()

'/content'

In [7]:
sys.path.append(str(root_path)) # Add the directory to the path
from utils import prepare_hdf5_data,get_data_paths_ordered,predict_on_hdf5,OpenWorldLabelEncoder

In [8]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [9]:
model_path=f'{data_path}/models/cnn_lstm_cache.keras'
le_dir=f'{data_path}/models/label_encoder_100.pkl'
model0 = tf.keras.models.load_model(model_path)
with open(le_dir, 'rb') as f:
    le = pickle.load(f)

ValueError: File not found: filepath=/content/drive/MyDrive/Colab Notebooks/concept-drift-online//data/dynamic_browser_version/network/models/cnn_lstm_cache.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
ordered_paths=get_data_paths_ordered(data_path)
batch=256


In [ ]:

results={}
for path in tqdm(ordered_paths):
    curr_date_i=path.stem
    test_path_i = (path/'test.h5').as_posix()
    pred_i=predict_on_hdf5(test_path_i,model0,le,batch_size=32)
    with h5py.File(test_path_i, 'r') as hdf:
      y_testi = le.transform((hdf[f'site_name'][:]).reshape(-1,1)).toarray()
    acci=(y_testi.argmax(axis=1)==pred_i.argmax(axis=1)).mean()
    results.update({curr_date_i:acci})
    print(f'accuracy: {acci}')

  0%|          | 0/7 [00:00<?, ?it/s]

32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step


 14%|█▍        | 1/7 [00:07<00:42,  7.05s/it]

accuracy: 0.917
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


 29%|██▊       | 2/7 [00:10<00:23,  4.79s/it]

...

accuracy: 0.718
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


 86%|████████▌ | 6/7 [00:21<00:02,  2.99s/it]

accuracy: 0.704
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


100%|██████████| 7/7 [00:24<00:00,  3.43s/it]

accuracy: 0.673


In [ ]:
results={}
batch_size=256
non_sensitive_label=69
owle = OpenWorldLabelEncoder(le,non_sensitive_label+1)

ow_model_path=f'{data_path}/models/cnn_lstm_ow.keras'
model_ow = tf.keras.models.load_model(ow_model_path)
for path in tqdm(ordered_paths):
    curr_date_i=path.stem
    test_path_i = (path/'test.h5').as_posix()
    test_datasets_i=prepare_hdf5_data(test_path_i,owle,batch_size=batch,sample_validation=-1)['data']
    all_predictions = []
    all_true_labels = []
    for features, labels in test_datasets_i:
        batch_predictions = model_ow.predict_on_batch(features)
        all_predictions.append(batch_predictions)
        all_true_labels.append(labels.numpy()) # Convert label tensor to NumPy array
    pred_i = np.concatenate(all_predictions, axis=0)
    y_test_aligned = np.concatenate(all_true_labels, axis=0)
    acci=(pred_i.argmax(axis=1)==y_test_aligned.argmax(axis=1)).mean()
    y_true_indices = y_test_aligned.argmax(axis=1)
    y_pred_indices = pred_i.argmax(axis=1)
    precision = precision_score(y_true_indices, y_pred_indices, labels=[non_sensitive_label], average='weighted')
    recall = recall_score(y_true_indices, y_pred_indices, labels=[non_sensitive_label], average='weighted')
    f2score = fbeta_score(y_true_indices, y_pred_indices, beta=2, labels=[non_sensitive_label], average='weighted')

    results.update({curr_date_i: {'accuracy': acci, 'precision': precision, 'recall': recall, 'f2score': f2score}})
    print(f'Date: {curr_date_i}, Accuracy: {acci}, Precision (Non-sensitive): {precision}, Recall (Non-sensitive): {recall}, F2-score (Non-sensitive): {f2score}')


 14%|█▍        | 1/7 [00:03<00:20,  3.38s/it]

Date: 2021_08_27, Accuracy: 0.911, Precision (Non-sensitive): 0.9076923076923076, Recall (Non-sensitive): 0.9516129032258065, F2-score (Non-sensitive): 0.9424920127795527


 29%|██▊       | 2/7 [00:04<00:09,  1.98s/it]

Date: 2021_09_05, Accuracy: 0.87, Precision (Non-sensitive): 0.8271954674220963, Recall (Non-sensitive): 0.9419354838709677, F2-score (Non-sensitive): 0.9165097300690521


 43%|████▎     | 3/7 [00:05<00:06,  1.66s/it]

...

Date: 2021_11_28, Accuracy: 0.713, Precision (Non-sensitive): 0.6674937965260546, Recall (Non-sensitive): 0.867741935483871, F2-score (Non-sensitive): 0.8186244674376141


 86%|████████▌ | 6/7 [00:09<00:01,  1.28s/it]

Date: 2022_01_23, Accuracy: 0.689, Precision (Non-sensitive): 0.6578313253012048, Recall (Non-sensitive): 0.8806451612903226, F2-score (Non-sensitive): 0.824773413897281


100%|██████████| 7/7 [00:10<00:00,  1.47s/it]

Date: 2022_02_21, Accuracy: 0.659, Precision (Non-sensitive): 0.5981735159817352, Recall (Non-sensitive): 0.8451612903225807, F2-score (Non-sensitive): 0.7806912991656734
